In [1]:
%pip install japanize-matplotlib

  Using cached japanize_matplotlib-1.1.3-py3-none-any.whl
Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'd:\Akash\Work\AI\model_evaluation\evaluation\Scripts\python.exe -m pip install --upgrade pip' command.


In [17]:
import pandas as pd
from matplotlib import cm
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import font_manager
import japanize_matplotlib
import os
import random
from scipy.signal import savgol_filter


def read_all_sheets(file_path):
# Load all sheets
    all_sheets = pd.read_excel(file_path, sheet_name=None)
    return all_sheets
def find_best_F1score_and_conf(df):
    max_values = {}
    for column in df.columns[1:]:
        max_value = df[column].max()
        corresponding_value = df[df[column] == max_value].iloc[0, 0]
        max_values[column] = (max_value, corresponding_value)
    return(max_values)
# Check the structure of the DataFrame
def draw_graph(all_sheets, save_path, ylabel_name, models_to_plot, required_sheets, models):
    os.makedirs(save_path, exist_ok=True)

    for sheet_name in required_sheets:
        df = all_sheets[sheet_name]
    # for sheet_name, df in all_sheets.items():
        max_values = find_best_F1score_and_conf(df)
        
        # Assume the first column is the x-axis
        x = df.iloc[:, 0]
        plt.figure(figsize=(12, 8))
        for model_name in models_to_plot:
            y_data = df[model_name]
            max_val = y_data.max()
            corr_val = df[y_data == max_val].iloc[0, 0]
            y_smooth = savgol_filter(y_data, window_length=11, polyorder=3)
            auc = np.trapz(y_smooth, x)
            color = models[model_name][1]
            plt.plot(x, y_smooth, linestyle='-', linewidth=2, color=color)
            # plt.plot(x, y_smooth, linestyle='-', linewidth=2, color=color, 
            #      label=models[model_name][0] + fr"({corr_val:.3f},{max_val:.3f}, AUC={auc:.3f})")
            # plt.annotate(f'{corr_val:.2f}', xy=(corr_val, max_val), xytext=(corr_val, max_val+0.02),
            #     textcoords='data', ha='center', va='bottom', fontsize=8,
            #     bbox=dict(boxstyle='round,pad=0.3', edgecolor='black', facecolor='white'))
        # # Adding title and labels

        # plt.plot([], [], color=(1.0, 0.0, 0.0), label=r"0.98")
        # plt.plot([], [], color=(0.0, 1.0, 0.0), label=r"0.94")
        # plt.plot([], [], color=(0.0, 0.0, 1.0), label=r"0.82")
        
        plt.plot([], [], color=(1.0, 0.0, 0.0), label=r"2%")
        plt.plot([], [], color=(0.0, 1.0, 0.0), label=r"8%")
        # plt.plot([], [], color=(0.0, 0.0, 1.0), label =r"背景なし")
        plt.title(sheet_name+str(0.98))
        plt.xlabel(df.columns[0])
        plt.ylabel(ylabel_name)
        
        # Adding legend
        plt.legend(loc='upper center', bbox_to_anchor=(0.5, -0.15), ncol=3)
        plt.grid(True)
        # Display the plot
        # plt.show()
        plt.savefig(fr"{save_path}\{sheet_name}_{ylabel_name}.png", bbox_inches='tight')

        plt.close()
def draw_ap_graph(all_sheets, save_path, models_to_plot, required_sheets, models):
    IOU = [ 0.5, 0.55, 0.6, 0.65, 0.7, 0.75, 0.8, 0.85, 0.9, 0.95]
    ylabel_name = r"AP"
    os.makedirs(save_path, exist_ok=True)

    for sheet_name in required_sheets:
        df = all_sheets[sheet_name]
        
        # Assume the first column is the x-axis
        x = IOU
        plt.figure(figsize=(12, 8))
        for model_name in models_to_plot:
            auc = np.trapz(df[model_name], x)
            color = models[model_name][1]
            plt.plot(x, df[model_name], linestyle='-', linewidth=2, color=color)
            # plt.plot(x, df[model_name], linestyle='-', linewidth=2, color=color, label=models[model_name][0] + fr" (avg={df[model_name].mean():.3f} AUC={auc:.3f})")
        plt.plot([], [], color=(1.0, 0.0, 0.0), label=r"2%")
        plt.plot([], [], color=(0.0, 1.0, 0.0), label=r"8%")
        # plt.plot([], [], color=(0.0, 0.0, 1.0), label =r"背景なし")
        plt.title(sheet_name+str(0.98))
        plt.xlabel(r"IOU")
        plt.ylabel(ylabel_name)
        
        # Adding legend
        plt.legend(loc='upper center', bbox_to_anchor=(0.5, -0.15), ncol=3)
        
        # Display the plot
        # plt.show()
        plt.grid(True)
        plt.savefig(fr"{save_path}\{sheet_name}_{ylabel_name}.png", bbox_inches='tight')

        plt.close()
    
def generate_random_colors(num_colors, seed=12):
    random.seed(seed)
    colors = []
    for _ in range(num_colors):
        rgb = (random.randint(0, 255), random.randint(0, 255), random.randint(0, 255))
        color = tuple([x / 255.0 for x in rgb])
        colors.append(color)
    return colors
matome_doc_dict = {
    r"F1":                  (r"matome_confidence_F1_curve.xlsx",r"F1"),
    r"Precision":           (r"matome_confidence_precision_curve.xlsx",r"Precision"),
    r"Recall":              (r"matome_confidence_recall.xlsx",r"Recall"),
    r"Precision_Recall":    (r"matome_precision_recall.xlsx",r"Precision"),
}
required_sheets = [
    r"ぼやけされている人",
    r"運転している",
    r"顔が見えない運転者の画像",
    r"結合データセット",
    r"屈んでいるポーズ",
    r"座っているポーズ",
    r"遮蔽されている人",
    r"体の部分",
    r"特徴がよくみえない画像",
    # r"背景",
    r"箱を持っている·運んでいるポーズ",
    r"立っているポーズ",

]

#generate the total number of colors
colors = generate_random_colors(30)

Dataset 1(2%)	vs	dataset9  
Dataset 2(8%)	vs	dataset9  
Dataset 3(2%)	vs	dataset13  
Dataset 4(8%)	vs	dataset13  
Dataset 5(2%)	vs	dataset17  
Dataset 6(8%)	vs	
dataset17

In [14]:

models = {
    #全体的の追加データと背景の影響検査
    r'240712124035_runs_train_2024_07_dataset_1': (r"1_HBI_7_0.98+tsuika_0.98_equal+bg_2_equal", colors[0]),
    r'240712133342_runs_train_2024_07_dataset_2': (r"2_HBI_7_0.98+tsuika_0.98_equal+bg_8_equal", colors[1]),
    r'240712113819_runs_train_2024_07_dataset_3': (r"3_HBI_7_0.94+tsuika_0.94_equal+bg_2_equal", colors[2]),
    r'240712230805_runs_train_2024_07_dataset_4': (r"4_HBI_7_0.94+tsuika_0.94_equal+bg_8_equal", colors[3]),
    r'240712170233_runs_train_2024_07_dataset_5': (r"5_HBI_7_0.82+tsuika_0.82_equal+bg_2_equal", colors[4]),
    r'240712172233_runs_train_2024_07_dataset_6': (r"6_HBI_7_0.82+tsuika_0.82_equal+bg_8_equal", colors[5]),
    #HBI_202403_7のPureデータと背景の影響検査
    r'240713095143_runs_train_2024_07_dataset_7': (r"7_HBI_7_pure+tsuika_NIL+bg_8_equal", colors[6]),
    r'240713091546_runs_train_2024_07_dataset_8': (r"8_HBI_7_pure+tsuika_NIL+bg_2_equal", colors[7]),
    #追加データの影響検査
    r'240713085856_runs_train_2024_07_dataset_9':  (r"9_HBI_7_0.98+tsuika_0.98_equal+bg_NIL", colors[8]),
    r'240713175733_runs_train_2024_07_dataset_10': (r"10_HBI_7_0.94+tsuika_0.98_equal+bg_NIL", colors[9]),
    r'240713131827_runs_train_2024_07_dataset_11': (r"11_HBI_7_0.82+tsuika_0.98_equal+bg_NIL", colors[10]),
    r'240713193726_runs_train_2024_07_dataset_12': (r"12_HBI_7_0.98+tsuika_0.94_equal+bg_NIL", colors[11]),
    r'240714074724_runs_train_2024_07_dataset_13': (r"13_HBI_7_0.94+tsuika_0.94_equal+bg_NIL", colors[12]),
    r'240714023654_runs_train_2024_07_dataset_14': (r"14_HBI_7_0.82+tsuika_0.94_equal+bg_NIL", colors[13]),
    r'240714090311_runs_train_2024_07_dataset_15': (r"15_HBI_7_0.98+tsuika_0.82_equal_as_BG", colors[14]),
    r'240714162405_runs_train_2024_07_dataset_16': (r"16_HBI_7_0.94+tsuika_0.82_equal_as_BG", colors[15]),
    r'240714112613_runs_train_2024_07_dataset_17': (r"17_HBI_7_0.82+tsuika_0.82_equal_as_BG", colors[16]),
    #背景の影響検査
    r'240714195951_runs_train_2024_07_dataset_18': (r"18_HBI_7_0.98+tsuika_NIL+bg_2_equal", colors[17]),
    r'240715074036_runs_train_2024_07_dataset_19': (r"19_HBI_7_0.94+tsuika_NIL+bg_2_equal", colors[18]),
    r'240715023142_runs_train_2024_07_dataset_20': (r"20_HBI_7_0.82+tsuika_NIL+bg_2_equal", colors[19]),
    r'240715093419_runs_train_2024_07_dataset_21': (r"21_HBI_7_0.98+tsuika_NIL+bg_8_equal", colors[20]),
    r'240715153347_runs_train_2024_07_dataset_22': (r"22_HBI_7_0.94+tsuika_NIL+bg_8_equal", colors[21]),
    r'240715121825_runs_train_2024_07_dataset_23': (r"23_HBI_7_0.82+tsuika_NIL+bg_8_equal", colors[22]),
    #corrected_models 
    r'240717192634_runs_train_2024_07_dataset_15_2': (r"15_2_HBI_7_0.98+tsuika_0.82_equal+bg_NIL", colors[23]),
    r'240717183312_runs_train_2024_07_dataset_16_2': (r"16_2_HBI_7_0.94+tsuika_0.82_equal+bg_NIL", colors[24]),
    r'240717133019_runs_train_2024_07_dataset_17_2': (r"17_2_HBI_7_0.82+tsuika_0.82_equal+bg_NIL", colors[25]),
    r'40525071329_runs_train_240409153123_runs_train_HBI+追加_クリーニング後_２月_ランダム_類似性_0_94':(r"HBI_7_0.94+tsuika_NIL+bg_0.94", colors[26]),
    r'240526021439_runs_train_240409153123_runs_train_HBI+追加_クリーニング後_２月_ランダム_類似性_0_98':(r"HBI_7_0.98+tsuika_NIL+bg_0.98", colors[27]),
    r'240613182512_runs_train_HBI+追加_２月アノテーション_ランダムsplit_類似性インデックス_0_82':(r"HBI_7_0.82+tsuika_NIL+bg_0.82", colors[28]),
    r"240614145641_runs_train_202403_7_BGs_2percent":(r"HBI_pure+tsuika_NIL+bg_2", colors[29]),
}


models_to_plot = [
    #全体的の追加データと背景の影響検査
    r'240712124035_runs_train_2024_07_dataset_1', 
    r'240712133342_runs_train_2024_07_dataset_2', 
    r'240712113819_runs_train_2024_07_dataset_3', 
    r'240712230805_runs_train_2024_07_dataset_4', 
    r'240712170233_runs_train_2024_07_dataset_5', 
    r'240712172233_runs_train_2024_07_dataset_6', 
    #HBI_202403_7のPureデータと背景の影響検査
    # r'240713095143_runs_train_2024_07_dataset_7',
    # r'240713091546_runs_train_2024_07_dataset_8',
     #追加データの影響検査
    # r'240713085856_runs_train_2024_07_dataset_9', 
    # r'240713175733_runs_train_2024_07_dataset_10',
    # r'240713131827_runs_train_2024_07_dataset_11',
    # r'240713193726_runs_train_2024_07_dataset_12',
    # r'240714074724_runs_train_2024_07_dataset_13',
    # r'240714023654_runs_train_2024_07_dataset_14',
    # r'240714090311_runs_train_2024_07_dataset_15',
    # r'240714162405_runs_train_2024_07_dataset_16',
    # r'240714112613_runs_train_2024_07_dataset_17',
    #背景の影響検査
    r'240714195951_runs_train_2024_07_dataset_18',
    r'240715074036_runs_train_2024_07_dataset_19',
    r'240715023142_runs_train_2024_07_dataset_20',
    r'240715093419_runs_train_2024_07_dataset_21',
    r'240715153347_runs_train_2024_07_dataset_22',
    r'240715121825_runs_train_2024_07_dataset_23',
    #corrected_models 
    # r'240717192634_runs_train_2024_07_dataset_15_2',
    # r'240717183312_runs_train_2024_07_dataset_16_2',
    # r'240717133019_runs_train_2024_07_dataset_17_2',
    # r"240614145641_runs_train_202403_7_BGs_2percent",

    # r'240525071329_runs_train_240409153123_runs_train_HBI+追加_クリーニング後_２月_ランダム_類似性_0_94',
    # r'240526021439_runs_train_240409153123_runs_train_HBI+追加_クリーニング後_２月_ランダム_類似性_0_98',
    # r'240613182512_runs_train_HBI+追加_２月アノテーション_ランダムsplit_類似性インデックス_0_82',
]



In [178]:
models_to_plot = [
    #全体的の追加データと背景の影響検査
    r'240713085856_runs_train_2024_07_dataset_9', 
    r'240712124035_runs_train_2024_07_dataset_1', 
    r'240712133342_runs_train_2024_07_dataset_2', 
    # r'240714074724_runs_train_2024_07_dataset_13',
    # r'240712113819_runs_train_2024_07_dataset_3', 
    # r'240712230805_runs_train_2024_07_dataset_4', 
    # r'240717133019_runs_train_2024_07_dataset_17_2',
    # r'240712170233_runs_train_2024_07_dataset_5', 
    # r'240712172233_runs_train_2024_07_dataset_6', 
    #HBI_202403_7のPureデータと背景の影響検査
    # r'240713095143_runs_train_2024_07_dataset_7',
    # r'240713091546_runs_train_2024_07_dataset_8',
     #追加データの影響検査
    # r'240713175733_runs_train_2024_07_dataset_10',
    # r'240713131827_runs_train_2024_07_dataset_11',
    # r'240713193726_runs_train_2024_07_dataset_12',
    # r'240714023654_runs_train_2024_07_dataset_14',
    # r'240714090311_runs_train_2024_07_dataset_15',
    # r'240714162405_runs_train_2024_07_dataset_16',
    # r'240714112613_runs_train_2024_07_dataset_17',
    #背景の影響検査
    # r'240714195951_runs_train_2024_07_dataset_18',
    # r'240715074036_runs_train_2024_07_dataset_19',
    # r'240715023142_runs_train_2024_07_dataset_20',
    # r'240715093419_runs_train_2024_07_dataset_21',
    # r'240715153347_runs_train_2024_07_dataset_22',
    # r'240715121825_runs_train_2024_07_dataset_23',
    #corrected_models 
    # r'240717192634_runs_train_2024_07_dataset_15_2',
    # r'240717183312_runs_train_2024_07_dataset_16_2',
    # r"240614145641_runs_train_202403_7_BGs_2percent",

    # r'240525071329_runs_train_240409153123_runs_train_HBI+追加_クリーニング後_２月_ランダム_類似性_0_94',
    # r'240526021439_runs_train_240409153123_runs_train_HBI+追加_クリーニング後_２月_ランダム_類似性_0_98',
    # r'240613182512_runs_train_HBI+追加_２月アノテーション_ランダムsplit_類似性インデックス_0_82',
]
models = {
    #全体的の追加データと背景の影響検査
    r'240712124035_runs_train_2024_07_dataset_1': (r"2%", (1.0, 0.0, 0.0)),
    r'240712133342_runs_train_2024_07_dataset_2': (r"8%", (0.0, 1.0, 0.0)),
    # r'240712113819_runs_train_2024_07_dataset_3': (r"2%", (1.0, 0.0, 0.0)),
    # r'240712230805_runs_train_2024_07_dataset_4': (r"8%", (0.0, 1.0, 0.0)),
    # r'240712170233_runs_train_2024_07_dataset_5': (r"2%", (1.0, 0.0, 0.0)),
    # r'240712172233_runs_train_2024_07_dataset_6': (r"8%", (0.0, 1.0, 0.0)),
    #HBI_202403_7のPureデータと背景の影響検査
    # r'240713095143_runs_train_2024_07_dataset_7': (r"7_HBI_7_pure+tsuika_NIL+bg_8_equal", colors[6]),
    # r'240713091546_runs_train_2024_07_dataset_8': (r"8_HBI_7_pure+tsuika_NIL+bg_2_equal", colors[7]),
    #追加データの影響検査
    r'240713085856_runs_train_2024_07_dataset_9':  (r"背景なし", (0.0, 0.0, 1.0)),
    # r'240713175733_runs_train_2024_07_dataset_10': (r"10_HBI_7_0.94+tsuika_0.98_equal+bg_NIL", (0.0, 1.0, 0.0)),
    # r'240713131827_runs_train_2024_07_dataset_11': (r"11_HBI_7_0.82+tsuika_0.98_equal+bg_NIL", (0.0, 0.0, 1.0)),
    # r'240713193726_runs_train_2024_07_dataset_12': (r"12_HBI_7_0.98+tsuika_0.94_equal+bg_NIL", (1.0, 0.0, 0.0)),
    # r'240714074724_runs_train_2024_07_dataset_13': (r"背景なし", (0.0, 0.0, 1.0)),
    # r'240714023654_runs_train_2024_07_dataset_14': (r"14_HBI_7_0.82+tsuika_0.94_equal+bg_NIL", (0.0, 0.0, 1.0)),
    # r'240714090311_runs_train_2024_07_dataset_15': (r"15_HBI_7_0.98+tsuika_0.82_equal_as_BG", (1.0, 0.0, 0.0)),
    # r'240714162405_runs_train_2024_07_dataset_16': (r"16_HBI_7_0.94+tsuika_0.82_equal_as_BG", (0.0, 1.0, 0.0)),
    # r'240714112613_runs_train_2024_07_dataset_17': (r"17_HBI_7_0.82+tsuika_0.82_equal_as_BG", (0.0, 0.0, 1.0)),
    #背景の影響検査
    # r'240714195951_runs_train_2024_07_dataset_18': (r"18_HBI_7_0.98+tsuika_NIL+bg_2_equal", (1.0, 0.0, 0.0)),
    # r'240715074036_runs_train_2024_07_dataset_19': (r"19_HBI_7_0.94+tsuika_NIL+bg_2_equal", (1.0, 0.0, 0.0)),
    # r'240715023142_runs_train_2024_07_dataset_20': (r"20_HBI_7_0.82+tsuika_NIL+bg_2_equal", (1.0, 0.0, 0.0)),
    # r'240715093419_runs_train_2024_07_dataset_21': (r"21_HBI_7_0.98+tsuika_NIL+bg_8_equal", (0.0, 1.0, 0.0)),
    # r'240715153347_runs_train_2024_07_dataset_22': (r"22_HBI_7_0.94+tsuika_NIL+bg_8_equal", (0.0, 1.0, 0.0)),
    # r'240715121825_runs_train_2024_07_dataset_23': (r"23_HBI_7_0.82+tsuika_NIL+bg_8_equal", (0.0, 1.0, 0.0)),
    #corrected_models 
    # r'240717192634_runs_train_2024_07_dataset_15_2': (r"15_2_HBI_7_0.98+tsuika_0.82_equal+bg_NIL", (1.0, 0.0, 0.0)),
    # r'240717183312_runs_train_2024_07_dataset_16_2': (r"16_2_HBI_7_0.94+tsuika_0.82_equal+bg_NIL", (0.0, 1.0, 0.0)),
    # r'240717133019_runs_train_2024_07_dataset_17_2': (r"背景なし", (0.0, 0.0, 1.0)),
    # r'40525071329_runs_train_240409153123_runs_train_HBI+追加_クリーニング後_２月_ランダム_類似性_0_94':(r"HBI_7_0.94+tsuika_NIL+bg_0.94", colors[26]),
    # r'240526021439_runs_train_240409153123_runs_train_HBI+追加_クリーニング後_２月_ランダム_類似性_0_98':(r"HBI_7_0.98+tsuika_NIL+bg_0.98", colors[27]),
    # r'240613182512_runs_train_HBI+追加_２月アノテーション_ランダムsplit_類似性インデックス_0_82':(r"HBI_7_0.82+tsuika_NIL+bg_0.82", colors[28]),
    # r"240614145641_runs_train_202403_7_BGs_2percent":(r"HBI_pure+tsuika_NIL+bg_2", colors[29]),
}

In [98]:
### HBI


models_to_plot = [
    #全体的の追加データと背景の影響検査
    r'240712124035_runs_train_2024_07_dataset_1', 
    r'240712133342_runs_train_2024_07_dataset_2', 
    r'240712113819_runs_train_2024_07_dataset_3', 
    r'240712230805_runs_train_2024_07_dataset_4', 
    r'240712170233_runs_train_2024_07_dataset_5', 
    r'240712172233_runs_train_2024_07_dataset_6', 
    #HBI_202403_7のPureデータと背景の影響検査
    # r'240713095143_runs_train_2024_07_dataset_7',
    # r'240713091546_runs_train_2024_07_dataset_8',
     #追加データの影響検査
    r'240713085856_runs_train_2024_07_dataset_9', 
    r'240713175733_runs_train_2024_07_dataset_10',
    r'240713131827_runs_train_2024_07_dataset_11',
    r'240713193726_runs_train_2024_07_dataset_12',
    r'240714074724_runs_train_2024_07_dataset_13',
    r'240714023654_runs_train_2024_07_dataset_14',
    r'240714090311_runs_train_2024_07_dataset_15',
    r'240714162405_runs_train_2024_07_dataset_16',
    r'240714112613_runs_train_2024_07_dataset_17',
    #背景の影響検査
    r'240714195951_runs_train_2024_07_dataset_18',
    r'240715074036_runs_train_2024_07_dataset_19',
    r'240715023142_runs_train_2024_07_dataset_20',
    r'240715093419_runs_train_2024_07_dataset_21',
    r'240715153347_runs_train_2024_07_dataset_22',
    r'240715121825_runs_train_2024_07_dataset_23',
    #corrected_models 
    r'240717192634_runs_train_2024_07_dataset_15_2',
    r'240717183312_runs_train_2024_07_dataset_16_2',
    r'240717133019_runs_train_2024_07_dataset_17_2',
    # r"240614145641_runs_train_202403_7_BGs_2percent",

    # r'240525071329_runs_train_240409153123_runs_train_HBI+追加_クリーニング後_２月_ランダム_類似性_0_94',
    # r'240526021439_runs_train_240409153123_runs_train_HBI+追加_クリーニング後_２月_ランダム_類似性_0_98',
    # r'240613182512_runs_train_HBI+追加_２月アノテーション_ランダムsplit_類似性インデックス_0_82',
]
models = {
    #全体的の追加データと背景の影響検査
    r'240712124035_runs_train_2024_07_dataset_1': (r"1_HBI_7_0.98+tsuika_0.98_equal+bg_2_equal", (1.0, 0.0, 0.0)),
    r'240712133342_runs_train_2024_07_dataset_2': (r"2_HBI_7_0.98+tsuika_0.98_equal+bg_8_equal", (1.0, 0.0, 0.0)),
    r'240712113819_runs_train_2024_07_dataset_3': (r"3_HBI_7_0.94+tsuika_0.94_equal+bg_2_equal", (0.0, 1.0, 0.0)),
    r'240712230805_runs_train_2024_07_dataset_4': (r"4_HBI_7_0.94+tsuika_0.94_equal+bg_8_equal", (0.0, 1.0, 0.0)),
    r'240712170233_runs_train_2024_07_dataset_5': (r"5_HBI_7_0.82+tsuika_0.82_equal+bg_2_equal", (0.0, 0.0, 1.0)),
    r'240712172233_runs_train_2024_07_dataset_6': (r"6_HBI_7_0.82+tsuika_0.82_equal+bg_8_equal", (0.0, 0.0, 1.0)),
    #HBI_202403_7のPureデータと背景の影響検査
    # r'240713095143_runs_train_2024_07_dataset_7': (r"7_HBI_7_pure+tsuika_NIL+bg_8_equal", colors[6]),
    # r'240713091546_runs_train_2024_07_dataset_8': (r"8_HBI_7_pure+tsuika_NIL+bg_2_equal", colors[7]),
    #追加データの影響検査
    r'240713085856_runs_train_2024_07_dataset_9':  (r"9_HBI_7_0.98+tsuika_0.98_equal+bg_NIL", (1.0, 0.0, 0.0)),
    r'240713175733_runs_train_2024_07_dataset_10': (r"10_HBI_7_0.94+tsuika_0.98_equal+bg_NIL", (0.0, 1.0, 0.0)),
    r'240713131827_runs_train_2024_07_dataset_11': (r"11_HBI_7_0.82+tsuika_0.98_equal+bg_NIL", (0.0, 0.0, 1.0)),
    r'240713193726_runs_train_2024_07_dataset_12': (r"12_HBI_7_0.98+tsuika_0.94_equal+bg_NIL", (1.0, 0.0, 0.0)),
    r'240714074724_runs_train_2024_07_dataset_13': (r"13_HBI_7_0.94+tsuika_0.94_equal+bg_NIL", (0.0, 1.0, 0.0)),
    r'240714023654_runs_train_2024_07_dataset_14': (r"14_HBI_7_0.82+tsuika_0.94_equal+bg_NIL", (0.0, 0.0, 1.0)),
    r'240714090311_runs_train_2024_07_dataset_15': (r"15_HBI_7_0.98+tsuika_0.82_equal_as_BG", (1.0, 0.0, 0.0)),
    r'240714162405_runs_train_2024_07_dataset_16': (r"16_HBI_7_0.94+tsuika_0.82_equal_as_BG", (0.0, 1.0, 0.0)),
    r'240714112613_runs_train_2024_07_dataset_17': (r"17_HBI_7_0.82+tsuika_0.82_equal_as_BG", (0.0, 0.0, 1.0)),
    #背景の影響検査
    r'240714195951_runs_train_2024_07_dataset_18': (r"18_HBI_7_0.98+tsuika_NIL+bg_2_equal", (1.0, 0.0, 0.0)),
    r'240715074036_runs_train_2024_07_dataset_19': (r"19_HBI_7_0.94+tsuika_NIL+bg_2_equal", (0.0, 1.0, 0.0)),
    r'240715023142_runs_train_2024_07_dataset_20': (r"20_HBI_7_0.82+tsuika_NIL+bg_2_equal", (0.0, 0.0, 1.0)),
    r'240715093419_runs_train_2024_07_dataset_21': (r"21_HBI_7_0.98+tsuika_NIL+bg_8_equal", (1.0, 0.0, 0.0)),
    r'240715153347_runs_train_2024_07_dataset_22': (r"22_HBI_7_0.94+tsuika_NIL+bg_8_equal", (0.0, 1.0, 0.0)),
    r'240715121825_runs_train_2024_07_dataset_23': (r"23_HBI_7_0.82+tsuika_NIL+bg_8_equal", (0.0, 0.0, 1.0)),
    #corrected_models 
    r'240717192634_runs_train_2024_07_dataset_15_2': (r"15_2_HBI_7_0.98+tsuika_0.82_equal+bg_NIL", (1.0, 0.0, 0.0)),
    r'240717183312_runs_train_2024_07_dataset_16_2': (r"16_2_HBI_7_0.94+tsuika_0.82_equal+bg_NIL", (0.0, 1.0, 0.0)),
    r'240717133019_runs_train_2024_07_dataset_17_2': (r"17_2_HBI_7_0.82+tsuika_0.82_equal+bg_NIL", (0.0, 0.0, 1.0)),
    r'40525071329_runs_train_240409153123_runs_train_HBI+追加_クリーニング後_２月_ランダム_類似性_0_94':(r"HBI_7_0.94+tsuika_NIL+bg_0.94", colors[26]),
    r'240526021439_runs_train_240409153123_runs_train_HBI+追加_クリーニング後_２月_ランダム_類似性_0_98':(r"HBI_7_0.98+tsuika_NIL+bg_0.98", colors[27]),
    r'240613182512_runs_train_HBI+追加_２月アノテーション_ランダムsplit_類似性インデックス_0_82':(r"HBI_7_0.82+tsuika_NIL+bg_0.82", colors[28]),
    r"240614145641_runs_train_202403_7_BGs_2percent":(r"HBI_pure+tsuika_NIL+bg_2", colors[29]),
}

In [18]:
### BG


models_to_plot = [
    #全体的の追加データと背景の影響検査
    r'240712124035_runs_train_2024_07_dataset_1', 
    r'240712133342_runs_train_2024_07_dataset_2', 
    r'240712113819_runs_train_2024_07_dataset_3', 
    r'240712230805_runs_train_2024_07_dataset_4', 
    r'240712170233_runs_train_2024_07_dataset_5', 
    r'240712172233_runs_train_2024_07_dataset_6', 
    #HBI_202403_7のPureデータと背景の影響検査
    r'240713095143_runs_train_2024_07_dataset_7',
    r'240713091546_runs_train_2024_07_dataset_8',
     #追加データの影響検査
    # r'240713085856_runs_train_2024_07_dataset_9', 
    # r'240713175733_runs_train_2024_07_dataset_10',
    # r'240713131827_runs_train_2024_07_dataset_11',
    # r'240713193726_runs_train_2024_07_dataset_12',
    # r'240714074724_runs_train_2024_07_dataset_13',
    # r'240714023654_runs_train_2024_07_dataset_14',
    # r'240714090311_runs_train_2024_07_dataset_15',
    # r'240714162405_runs_train_2024_07_dataset_16',
    # r'240714112613_runs_train_2024_07_dataset_17',
    #背景の影響検査
    r'240714195951_runs_train_2024_07_dataset_18',
    r'240715074036_runs_train_2024_07_dataset_19',
    r'240715023142_runs_train_2024_07_dataset_20',
    r'240715093419_runs_train_2024_07_dataset_21',
    r'240715153347_runs_train_2024_07_dataset_22',
    r'240715121825_runs_train_2024_07_dataset_23',
    #corrected_models 
    # r'240717192634_runs_train_2024_07_dataset_15_2',
    # r'240717183312_runs_train_2024_07_dataset_16_2',
    # r'240717133019_runs_train_2024_07_dataset_17_2',
    # r"240614145641_runs_train_202403_7_BGs_2percent",

    # r'240525071329_runs_train_240409153123_runs_train_HBI+追加_クリーニング後_２月_ランダム_類似性_0_94',
    # r'240526021439_runs_train_240409153123_runs_train_HBI+追加_クリーニング後_２月_ランダム_類似性_0_98',
    # r'240613182512_runs_train_HBI+追加_２月アノテーション_ランダムsplit_類似性インデックス_0_82',
]
models = {
    #全体的の追加データと背景の影響検査
    r'240712124035_runs_train_2024_07_dataset_1': (r"1_HBI_7_0.98+tsuika_0.98_equal+bg_2_equal", (1.0, 0.0, 0.0)),
    r'240712133342_runs_train_2024_07_dataset_2': (r"2_HBI_7_0.98+tsuika_0.98_equal+bg_8_equal", (0.0, 1.0, 0.0)),
    r'240712113819_runs_train_2024_07_dataset_3': (r"3_HBI_7_0.94+tsuika_0.94_equal+bg_2_equal", (1.0, 0.0, 0.0)),
    r'240712230805_runs_train_2024_07_dataset_4': (r"4_HBI_7_0.94+tsuika_0.94_equal+bg_8_equal", (0.0, 1.0, 0.0)),
    r'240712170233_runs_train_2024_07_dataset_5': (r"5_HBI_7_0.82+tsuika_0.82_equal+bg_2_equal", (1.0, 0.0, 0.0)),
    r'240712172233_runs_train_2024_07_dataset_6': (r"6_HBI_7_0.82+tsuika_0.82_equal+bg_8_equal", (0.0, 1.0, 0.0)),
    #HBI_202403_7のPureデータと背景の影響検査
    r'240713095143_runs_train_2024_07_dataset_7': (r"7_HBI_7_pure+tsuika_NIL+bg_8_equal", (1.0, 0.0, 0.0)),
    r'240713091546_runs_train_2024_07_dataset_8': (r"8_HBI_7_pure+tsuika_NIL+bg_2_equal", (0.0, 1.0, 0.0)),
    #追加データの影響検査
    # r'240713085856_runs_train_2024_07_dataset_9':  (r"9_HBI_7_0.98+tsuika_0.98_equal+bg_NIL", (1.0, 0.0, 0.0)),
    # r'240713175733_runs_train_2024_07_dataset_10': (r"10_HBI_7_0.94+tsuika_0.98_equal+bg_NIL", (0.0, 1.0, 0.0)),
    # r'240713131827_runs_train_2024_07_dataset_11': (r"11_HBI_7_0.82+tsuika_0.98_equal+bg_NIL", (0.0, 0.0, 1.0)),
    # r'240713193726_runs_train_2024_07_dataset_12': (r"12_HBI_7_0.98+tsuika_0.94_equal+bg_NIL", (1.0, 0.0, 0.0)),
    # r'240714074724_runs_train_2024_07_dataset_13': (r"13_HBI_7_0.94+tsuika_0.94_equal+bg_NIL", (0.0, 1.0, 0.0)),
    # r'240714023654_runs_train_2024_07_dataset_14': (r"14_HBI_7_0.82+tsuika_0.94_equal+bg_NIL", (0.0, 0.0, 1.0)),
    # r'240714090311_runs_train_2024_07_dataset_15': (r"15_HBI_7_0.98+tsuika_0.82_equal_as_BG", (1.0, 0.0, 0.0)),
    # r'240714162405_runs_train_2024_07_dataset_16': (r"16_HBI_7_0.94+tsuika_0.82_equal_as_BG", (0.0, 1.0, 0.0)),
    # r'240714112613_runs_train_2024_07_dataset_17': (r"17_HBI_7_0.82+tsuika_0.82_equal_as_BG", (0.0, 0.0, 1.0)),
    #背景の影響検査
    r'240714195951_runs_train_2024_07_dataset_18': (r"18_HBI_7_0.98+tsuika_NIL+bg_2_equal", (1.0, 0.0, 0.0)),
    r'240715074036_runs_train_2024_07_dataset_19': (r"19_HBI_7_0.94+tsuika_NIL+bg_2_equal", (1.0, 0.0, 0.0)),
    r'240715023142_runs_train_2024_07_dataset_20': (r"20_HBI_7_0.82+tsuika_NIL+bg_2_equal", (1.0, 0.0, 0.0)),
    r'240715093419_runs_train_2024_07_dataset_21': (r"21_HBI_7_0.98+tsuika_NIL+bg_8_equal", (0.0, 1.0, 0.0)),
    r'240715153347_runs_train_2024_07_dataset_22': (r"22_HBI_7_0.94+tsuika_NIL+bg_8_equal", (0.0, 1.0, 0.0)),
    r'240715121825_runs_train_2024_07_dataset_23': (r"23_HBI_7_0.82+tsuika_NIL+bg_8_equal", (0.0, 1.0, 0.0)),
    #corrected_models 
    # r'240717192634_runs_train_2024_07_dataset_15_2': (r"15_2_HBI_7_0.98+tsuika_0.82_equal+bg_NIL", (1.0, 0.0, 0.0)),
    # r'240717183312_runs_train_2024_07_dataset_16_2': (r"16_2_HBI_7_0.94+tsuika_0.82_equal+bg_NIL", (0.0, 1.0, 0.0)),
    # r'240717133019_runs_train_2024_07_dataset_17_2': (r"17_2_HBI_7_0.82+tsuika_0.82_equal+bg_NIL", (0.0, 0.0, 1.0)),
    # r'40525071329_runs_train_240409153123_runs_train_HBI+追加_クリーニング後_２月_ランダム_類似性_0_94':(r"HBI_7_0.94+tsuika_NIL+bg_0.94", colors[26]),
    # r'240526021439_runs_train_240409153123_runs_train_HBI+追加_クリーニング後_２月_ランダム_類似性_0_98':(r"HBI_7_0.98+tsuika_NIL+bg_0.98", colors[27]),
    # r'240613182512_runs_train_HBI+追加_２月アノテーション_ランダムsplit_類似性インデックス_0_82':(r"HBI_7_0.82+tsuika_NIL+bg_0.82", colors[28]),
    # r"240614145641_runs_train_202403_7_BGs_2percent":(r"HBI_pure+tsuika_NIL+bg_2", colors[29]),
}

In [19]:
base_dir = r"E:\analysis\20240718"
matome_data_dir =os.path.join(base_dir,r"matome")
graph_save_dir = os.path.join(base_dir,r"graphs")
research_theme = r"newBG"

In [20]:
for key,value in matome_doc_dict.items():
    all_sheets = read_all_sheets(file_path = os.path.join(matome_data_dir,value[0]))
    draw_graph(all_sheets, os.path.join(graph_save_dir, research_theme, key), value[1], models_to_plot, required_sheets, models)

all_sheets = read_all_sheets(file_path = fr"{matome_data_dir}\matome_ap.xlsx")
draw_ap_graph(all_sheets, fr"{graph_save_dir}\{research_theme}\AP", models_to_plot, required_sheets, models)

## change the analysis

0.98

In [25]:
import pandas as pd
from matplotlib import cm
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import font_manager
import japanize_matplotlib
import os
import random
from scipy.signal import savgol_filter


def read_all_sheets(file_path):
# Load all sheets
    all_sheets = pd.read_excel(file_path, sheet_name=None)
    return all_sheets
def find_best_F1score_and_conf(df):
    max_values = {}
    for column in df.columns[1:]:
        max_value = df[column].max()
        corresponding_value = df[df[column] == max_value].iloc[0, 0]
        max_values[column] = (max_value, corresponding_value)
    return(max_values)
# Check the structure of the DataFrame
def draw_graph(all_sheets, save_path, ylabel_name, models_to_plot, required_sheets, models):
    os.makedirs(save_path, exist_ok=True)

    for sheet_name in required_sheets:
        df = all_sheets[sheet_name]
    # for sheet_name, df in all_sheets.items():
        max_values = find_best_F1score_and_conf(df)
        
        # Assume the first column is the x-axis
        x = df.iloc[:, 0]
        plt.figure(figsize=(12, 8))
        for model_name in models_to_plot:
            y_data = df[model_name]
            max_val = y_data.max()
            corr_val = df[y_data == max_val].iloc[0, 0]
            y_smooth = savgol_filter(y_data, window_length=11, polyorder=3)
            auc = np.trapz(y_smooth, x)
            color = models[model_name][1]
            plt.plot(x, y_smooth, linestyle='-', linewidth=2, color=color)
            # plt.plot(x, y_smooth, linestyle='-', linewidth=2, color=color, 
            #      label=models[model_name][0] + fr"({corr_val:.3f},{max_val:.3f}, AUC={auc:.3f})")
            # plt.annotate(f'{corr_val:.2f}', xy=(corr_val, max_val), xytext=(corr_val, max_val+0.02),
            #     textcoords='data', ha='center', va='bottom', fontsize=8,
            #     bbox=dict(boxstyle='round,pad=0.3', edgecolor='black', facecolor='white'))
        # # Adding title and labels

        plt.plot([], [], color=(1.0, 0.0, 0.0), label=r"2%")
        plt.plot([], [], color=(0.0, 1.0, 0.0), label=r"8%")
        plt.title(sheet_name) 
        plt.xlabel(df.columns[0])
        plt.ylabel(ylabel_name)
        
        # Adding legend
        plt.legend(loc='upper center', bbox_to_anchor=(0.5, -0.15), ncol=3)
        plt.grid(True)
        # Display the plot
        # plt.show()
        plt.savefig(fr"{save_path}\{sheet_name}_{ylabel_name}.png", bbox_inches='tight')

        plt.close()
def draw_ap_graph(all_sheets, save_path, models_to_plot, required_sheets, models):
    IOU = [ 0.5, 0.55, 0.6, 0.65, 0.7, 0.75, 0.8, 0.85, 0.9, 0.95]
    ylabel_name = r"AP"
    os.makedirs(save_path, exist_ok=True)

    for sheet_name in required_sheets:
        df = all_sheets[sheet_name]
        
        # Assume the first column is the x-axis
        x = IOU
        plt.figure(figsize=(12, 8))
        for model_name in models_to_plot:
            auc = np.trapz(df[model_name], x)
            color = models[model_name][1]
            plt.plot(x, df[model_name], linestyle='-', linewidth=2, color=color)
            # plt.plot(x, df[model_name], linestyle='-', linewidth=2, color=color, label=models[model_name][0] + fr" (avg={df[model_name].mean():.3f} AUC={auc:.3f})")
            # plt.plot(x, df[model_name], linestyle='-', linewidth=2, color=color, label=models[model_name][0] + fr" (avg={df[model_name].mean():.3f} AUC={auc:.3f})")
        plt.plot([], [], color=(1.0, 0.0, 0.0), label=r"2%")
        plt.plot([], [], color=(0.0, 1.0, 0.0), label=r"8%")
        plt.title(sheet_name)
        plt.xlabel(r"IOU")
        plt.ylabel(ylabel_name)
        
        # Adding legend
        plt.legend(loc='upper center', bbox_to_anchor=(0.5, -0.15), ncol=3)
        
        # Display the plot
        # plt.show()
        plt.grid(True)
        plt.savefig(fr"{save_path}\{sheet_name}_{ylabel_name}.png", bbox_inches='tight')

        plt.close()
    
def generate_random_colors(num_colors, seed=12):
    random.seed(seed)
    colors = []
    for _ in range(num_colors):
        rgb = (random.randint(0, 255), random.randint(0, 255), random.randint(0, 255))
        color = tuple([x / 255.0 for x in rgb])
        colors.append(color)
    return colors
matome_doc_dict = {
    r"F1":                  (r"matome_confidence_F1_curve.xlsx",r"F1"),
    r"Precision":           (r"matome_confidence_precision_curve.xlsx",r"Precision"),
    r"Recall":              (r"matome_confidence_recall.xlsx",r"Recall"),
    r"Precision_Recall":    (r"matome_precision_recall.xlsx",r"Precision"),
}

#generate the total number of colors
colors = generate_random_colors(30)

In [27]:
### BG


models_to_plot = [
    #全体的の追加データと背景の影響検査
    r'240712124035_runs_train_2024_07_dataset_1', 
    r'240712133342_runs_train_2024_07_dataset_2', 
    # r'240712113819_runs_train_2024_07_dataset_3', 
    # r'240712230805_runs_train_2024_07_dataset_4', 
    # r'240712170233_runs_train_2024_07_dataset_5', 
    # r'240712172233_runs_train_2024_07_dataset_6', 
    #HBI_202403_7のPureデータと背景の影響検査
    # r'240713095143_runs_train_2024_07_dataset_7',
    # r'240713091546_runs_train_2024_07_dataset_8',
     #追加データの影響検査
    r'240713085856_runs_train_2024_07_dataset_9', 
    # r'240713175733_runs_train_2024_07_dataset_10',
    # r'240713131827_runs_train_2024_07_dataset_11',
    # r'240713193726_runs_train_2024_07_dataset_12',
    # r'240714074724_runs_train_2024_07_dataset_13',
    # r'240714023654_runs_train_2024_07_dataset_14',
    # r'240714090311_runs_train_2024_07_dataset_15',
    # r'240714162405_runs_train_2024_07_dataset_16',
    # r'240714112613_runs_train_2024_07_dataset_17',
    #背景の影響検査
    # r'240714195951_runs_train_2024_07_dataset_18',
    # r'240715074036_runs_train_2024_07_dataset_19',
    # r'240715023142_runs_train_2024_07_dataset_20',
    # r'240715093419_runs_train_2024_07_dataset_21',
    # r'240715153347_runs_train_2024_07_dataset_22',
    # r'240715121825_runs_train_2024_07_dataset_23',
    #corrected_models 
    # r'240717192634_runs_train_2024_07_dataset_15_2',
    # r'240717183312_runs_train_2024_07_dataset_16_2',
    # r'240717133019_runs_train_2024_07_dataset_17_2',
    # r"240614145641_runs_train_202403_7_BGs_2percent",

    # r'240525071329_runs_train_240409153123_runs_train_HBI+追加_クリーニング後_２月_ランダム_類似性_0_94',
    # r'240526021439_runs_train_240409153123_runs_train_HBI+追加_クリーニング後_２月_ランダム_類似性_0_98',
    # r'240613182512_runs_train_HBI+追加_２月アノテーション_ランダムsplit_類似性インデックス_0_82',
]
models = {
    #全体的の追加データと背景の影響検査
    r'240712124035_runs_train_2024_07_dataset_1': (r"1_HBI_7_0.98+tsuika_0.98_equal+bg_2_equal", (1.0, 0.0, 0.0)),
    r'240712133342_runs_train_2024_07_dataset_2': (r"2_HBI_7_0.98+tsuika_0.98_equal+bg_8_equal", (0.0, 1.0, 0.0)),
    # r'240712113819_runs_train_2024_07_dataset_3': (r"3_HBI_7_0.94+tsuika_0.94_equal+bg_2_equal", (1.0, 0.0, 0.0)),
    # r'240712230805_runs_train_2024_07_dataset_4': (r"4_HBI_7_0.94+tsuika_0.94_equal+bg_8_equal", (0.0, 1.0, 0.0)),
    # r'240712170233_runs_train_2024_07_dataset_5': (r"5_HBI_7_0.82+tsuika_0.82_equal+bg_2_equal", (1.0, 0.0, 0.0)),
    # r'240712172233_runs_train_2024_07_dataset_6': (r"6_HBI_7_0.82+tsuika_0.82_equal+bg_8_equal", (0.0, 1.0, 0.0)),
    #HBI_202403_7のPureデータと背景の影響検査
    # r'240713095143_runs_train_2024_07_dataset_7': (r"7_HBI_7_pure+tsuika_NIL+bg_8_equal", colors[6]),
    # r'240713091546_runs_train_2024_07_dataset_8': (r"8_HBI_7_pure+tsuika_NIL+bg_2_equal", colors[7]),
    #追加データの影響検査
    r'240713085856_runs_train_2024_07_dataset_9':  (r"9_HBI_7_0.98+tsuika_0.98_equal+bg_NIL", (.0, 0.0, 1.0)),
    # r'240713175733_runs_train_2024_07_dataset_10': (r"10_HBI_7_0.94+tsuika_0.98_equal+bg_NIL", (0.0, 1.0, 0.0)),
    # r'240713131827_runs_train_2024_07_dataset_11': (r"11_HBI_7_0.82+tsuika_0.98_equal+bg_NIL", (0.0, 0.0, 1.0)),
    # r'240713193726_runs_train_2024_07_dataset_12': (r"12_HBI_7_0.98+tsuika_0.94_equal+bg_NIL", (1.0, 0.0, 0.0)),
    # r'240714074724_runs_train_2024_07_dataset_13': (r"13_HBI_7_0.94+tsuika_0.94_equal+bg_NIL", (0.0, 1.0, 0.0)),
    # r'240714023654_runs_train_2024_07_dataset_14': (r"14_HBI_7_0.82+tsuika_0.94_equal+bg_NIL", (0.0, 0.0, 1.0)),
    # r'240714090311_runs_train_2024_07_dataset_15': (r"15_HBI_7_0.98+tsuika_0.82_equal_as_BG", (1.0, 0.0, 0.0)),
    # r'240714162405_runs_train_2024_07_dataset_16': (r"16_HBI_7_0.94+tsuika_0.82_equal_as_BG", (0.0, 1.0, 0.0)),
    # r'240714112613_runs_train_2024_07_dataset_17': (r"17_HBI_7_0.82+tsuika_0.82_equal_as_BG", (0.0, 0.0, 1.0)),
    #背景の影響検査
    # r'240714195951_runs_train_2024_07_dataset_18': (r"18_HBI_7_0.98+tsuika_NIL+bg_2_equal", (1.0, 0.0, 0.0)),
    # r'240715074036_runs_train_2024_07_dataset_19': (r"19_HBI_7_0.94+tsuika_NIL+bg_2_equal", (1.0, 0.0, 0.0)),
    # r'240715023142_runs_train_2024_07_dataset_20': (r"20_HBI_7_0.82+tsuika_NIL+bg_2_equal", (1.0, 0.0, 0.0)),
    # r'240715093419_runs_train_2024_07_dataset_21': (r"21_HBI_7_0.98+tsuika_NIL+bg_8_equal", (0.0, 1.0, 0.0)),
    # r'240715153347_runs_train_2024_07_dataset_22': (r"22_HBI_7_0.94+tsuika_NIL+bg_8_equal", (0.0, 1.0, 0.0)),
    # r'240715121825_runs_train_2024_07_dataset_23': (r"23_HBI_7_0.82+tsuika_NIL+bg_8_equal", (0.0, 1.0, 0.0)),
    #corrected_models 
    # r'240717192634_runs_train_2024_07_dataset_15_2': (r"15_2_HBI_7_0.98+tsuika_0.82_equal+bg_NIL", (1.0, 0.0, 0.0)),
    # r'240717183312_runs_train_2024_07_dataset_16_2': (r"16_2_HBI_7_0.94+tsuika_0.82_equal+bg_NIL", (0.0, 1.0, 0.0)),
    # r'240717133019_runs_train_2024_07_dataset_17_2': (r"17_2_HBI_7_0.82+tsuika_0.82_equal+bg_NIL", (0.0, 0.0, 1.0)),
    # r'40525071329_runs_train_240409153123_runs_train_HBI+追加_クリーニング後_２月_ランダム_類似性_0_94':(r"HBI_7_0.94+tsuika_NIL+bg_0.94", colors[26]),
    # r'240526021439_runs_train_240409153123_runs_train_HBI+追加_クリーニング後_２月_ランダム_類似性_0_98':(r"HBI_7_0.98+tsuika_NIL+bg_0.98", colors[27]),
    # r'240613182512_runs_train_HBI+追加_２月アノテーション_ランダムsplit_類似性インデックス_0_82':(r"HBI_7_0.82+tsuika_NIL+bg_0.82", colors[28]),
    # r"240614145641_runs_train_202403_7_BGs_2percent":(r"HBI_pure+tsuika_NIL+bg_2", colors[29]),
}

In [28]:
base_dir = r"E:\analysis\20240718"
matome_data_dir =os.path.join(base_dir,r"matome")
graph_save_dir = os.path.join(base_dir,r"graphs")
research_theme = r"test"

for key,value in matome_doc_dict.items():
    all_sheets = read_all_sheets(file_path = os.path.join(matome_data_dir,value[0]))
    draw_graph(all_sheets, os.path.join(graph_save_dir, research_theme, key), value[1], models_to_plot, required_sheets, models)

all_sheets = read_all_sheets(file_path = fr"{matome_data_dir}\matome_ap.xlsx")
draw_ap_graph(all_sheets, fr"{graph_save_dir}\{research_theme}\AP", models_to_plot, required_sheets, models)